# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846799749123                   -0.70    4.8         
  2   -7.852313461555       -2.26       -1.53    1.0   33.6ms
  3   -7.852611188623       -3.53       -2.55    1.5   36.3ms
  4   -7.852645847190       -4.46       -2.85    2.5   46.1ms
  5   -7.852646416021       -6.25       -3.08    1.0   33.6ms
  6   -7.852646677617       -6.58       -3.99    1.0   33.4ms
  7   -7.852646686605       -8.05       -5.00    2.0   42.8ms
  8   -7.852646686724       -9.93       -5.45    1.8   40.8ms
  9   -7.852646686725      -12.23       -5.41    1.2   35.6ms
 10   -7.852646686730      -11.28       -6.59    1.0   35.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846863286995                   -0.70           4.8         
  2   -7.852528840533       -2.25       -1.63   0.80    2.0    317ms
  3   -7.852637908190       -3.96       -2.71   0.80    1.0   32.0ms
  4   -7.852646432835       -5.07       -3.32   0.80    2.0   40.1ms
  5   -7.852646675432       -6.62       -4.15   0.80    1.8   36.7ms
  6   -7.852646686327       -7.96       -4.73   0.80    1.8   40.2ms
  7   -7.852646686724       -9.40       -5.60   0.80    1.5   36.0ms
  8   -7.852646686730      -11.27       -7.29   0.80    1.8   38.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.378708e+01     3.955225e+00
 * time: 0.5028510093688965
     1     1.029400e+00     2.356305e+00
 * time: 0.7559418678283691
     2    -1.675675e+00     2.778110e+00
 * time: 0.7901740074157715
     3    -3.818062e+00     2.344338e+00
 * time: 0.9732339382171631
     4    -5.253718e+00     2.054170e+00
 * time: 1.0196619033813477
     5    -6.949119e+00     9.956327e-01
 * time: 1.072523832321167
     6    -7.122219e+00     1.182051e+00
 * time: 1.104691982269287
     7    -7.623162e+00     7.931614e-01
 * time: 1.1364359855651855
     8    -7.695074e+00     9.822504e-01
 * time: 1.1688408851623535
     9    -7.730835e+00     7.180154e-01
 * time: 1.2021379470825195
    10    -7.750379e+00     8.211569e-01
 * time: 1.2365059852600098
    11    -7.774729e+00     5.687294e-01
 * time: 1.2705349922180176
    12    -7.784415e+00     5.587531e-01
 * time: 1.3033828735351562
    13    -7.800241e+00     2.129523e-01
 * time: 1.335592031478

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846835829397                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645906894                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.10s
  3   -7.852646686730      -13.34       -7.25    216ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.9238702920446444e-7
|ρ_newton - ρ_scfv| = 6.225403246354861e-8
|ρ_newton - ρ_dm|   = 9.537871503061952e-10
